# Road base Coordinate Address 컬럼을 생성하여 데이터프레임에 추가

In [1]:
import pandas as pd
from pathlib import Path

In [6]:
# 현재 작업 폴더에서 CSV 파일 2개를 찾아 읽어오는 코드 블록
cwd = Path.cwd()
# 상대경로
file1 = cwd / "processed_data" / "df_roadCoord_API.csv"
file2 = cwd / "processed_data" / "only_geopy_result.csv"

df_roadCoord = pd.read_csv(file1)
df_only_geopy = pd.read_csv(file2)

print(f"읽은 파일 1: {file1}")
print(df_roadCoord.shape)
print(f"읽은 파일 2: {file2}")
print(df_only_geopy.shape)

읽은 파일 1: /home/super/workspace/donghyun/apart_prj/processed_data/df_roadCoord_API.csv
(637, 3)
읽은 파일 2: /home/super/workspace/donghyun/apart_prj/processed_data/only_geopy_result.csv
(10842, 3)


In [7]:
# 두 데이터프레임 각각에서 중복된 행 제거
df_roadCoord = df_roadCoord.drop_duplicates()
df_only_geopy = df_only_geopy.drop_duplicates()
print(f"roadCoord_API 중복 제거 후 shape: {df_roadCoord.shape}")
print(f"only_geopy_result 중복 제거 후 shape: {df_only_geopy.shape}")

roadCoord_API 중복 제거 후 shape: (637, 3)
only_geopy_result 중복 제거 후 shape: (3083, 3)


In [8]:
# only_geopy_result에서 lat과 lon이 0 인 행 shape
zero_lat_lon = df_only_geopy[(df_only_geopy['lat'] == 0) | (df_only_geopy['lon'] == 0)]
print(f"only_geopy_result에서 lat 또는 lon이 0인 행 개수: {zero_lat_lon.shape[0]}")

only_geopy_result에서 lat 또는 lon이 0인 행 개수: 648


In [10]:
# zero_lat_lon 데이터프레임의 address컬럼값을 참고하여 df_roadCoord에서 해당 address의 좌표값을 가져와서
# only_geopy_result의 lat, lon 컬럼값을 대체
updated = 0
missing = []

for index, row in zero_lat_lon.iterrows():
    address = row['address']
    matched = df_roadCoord.loc[df_roadCoord['address'] == address, ['lat', 'lon']]
    if matched.empty:
        # df_roadCoord에 해당 주소가 없는 경우 건너뜀(또는 필요시 별도 처리)
        missing.append(address)
        continue
    lat, lon = matched.values[0]
    df_only_geopy.loc[index, 'lat'] = lat
    df_only_geopy.loc[index, 'lon'] = lon
    updated += 1

print(f"좌표를 대체한 행 수: {updated}")
if missing:
    print(f"roadCoord에 없는 주소 수: {len(missing)} (예시 최대 10개): {missing[:10]}")

좌표를 대체한 행 수: 637
roadCoord에 없는 주소 수: 11 (예시 최대 10개): ['성호북4길', '성호북5길', '석전남4길', '석전남13길', '대원로60번길', '대원로90번길', '대원로104번길', '대원로64번길', '대원로86번길', '대원로82번길']


In [11]:
# df_only_geopy에서 여전히 lat 또는 lon이 0인 행 개수 확인
still_zero = df_only_geopy[(df_only_geopy['lat'] == 0) | (df_only_geopy['lon'] == 0)]
print(f"대체 후에도 lat 또는 lon이 0인 행 개수: {still_zero.shape[0]}")

대체 후에도 lat 또는 lon이 0인 행 개수: 11


In [12]:
# 최종적으로 확보한 도로명 coordinate 데이터 프레임 생성
df_final_roadCoord = df_only_geopy.copy()
df_final_roadCoord.shape

(3083, 3)

In [ ]:
# 최종데이터프레임 csv로 저장
df_final_roadCoord.to_csv('II-2. final_roadCoord_information.csv', index=False, encoding='utf-8-sig')